In [10]:
import time
import numpy as np
import tensorflow as tf
import reader_ibm


In [11]:
data_dir='C:/Users/kusha/OneDrive/Desktop/Task/Deep_Learning_Everyday/LSTM_IBM/simple-examples/simple-examples/data'

In [12]:
init_sclae=0.1
learning_rate=1.0
max_grad_norm=5
num_layers=2
num_steps=20
hidden_size_l1=256
hidden_size_l2=128
max_epoch_decay_lr=4
max_epochs=15
keep_prob=1
decay=0.5
batch_size=30
vocab_size=10000
embedding_vector_size=200
is_training=1



In [13]:
raw_data = reader_ibm.ptb_raw_data(data_path=data_dir)
train_data, valid_data, test_data, vocab, word_to_id = raw_data

In [83]:
embedding_layer=tf.keras.layers.Embedding(vocab_size,embedding_vector_size,input_shape=(num_steps,),trainable=True,name='embedding_vocab')

C:\Users\kusha\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [85]:
inputs=embedding_layer(_input_data)

In [86]:
inputs.shape

TensorShape([30, 20, 200])

In [28]:
lstm_cell_l1=tf.keras.layers.LSTMCell(hidden_size_l1)
lstm_cell_l2=tf.keras.layers.LSTMCell(hidden_size_l2)

In [30]:
stacked_lstm=tf.keras.layers.StackedRNNCells([lstm_cell_l1,lstm_cell_l2])

In [31]:
stacked_lstm

<StackedRNNCells name=stacked_rnn_cells, built=False>

In [32]:
layer=tf.keras.layers.RNN(stacked_lstm,[batch_size,num_steps],return_state=False,stateful=True,trainable=False)

In [33]:
init_state=tf.Variable(tf.zeros([batch_size,embedding_vector_size]),trainable=False)

In [35]:
layer.initial_state=init_state

In [36]:
layer.initial_state

<tf.Variable 'Variable:0' shape=(30, 200) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [37]:
layer.initial_state.shape

TensorShape([30, 200])

In [38]:
output=layer(inputs)

In [39]:
output.shape

TensorShape([30, 20, 128])

In [40]:
# reshaping the outputs from shape 30x20x128 to 30x20x10000

dense=tf.keras.layers.Dense(vocab_size)

In [41]:
logits_outputs=dense(output)

In [42]:
print(logits_outputs.shape)

(30, 20, 10000)


In [43]:
activation=tf.keras.layers.Activation('softmax')


In [44]:
output_words_prob=activation(logits_outputs)


In [45]:
output_words_prob.shape

TensorShape([30, 20, 10000])

In [46]:
prediction=np.argmax(output_words_prob[0,0:num_steps],axis=1)

In [47]:
prediction.shape

(20,)

In [48]:
_targets[0].shape

(20,)

In [49]:
def crossentropy(target,predicted):
    return tf.keras.losses.sparse_categorical_crossentropy(target,predicted)

loss=crossentropy(_targets,output_words_prob)

In [50]:
loss

<tf.Tensor: shape=(30, 20), dtype=float32, numpy=
array([[9.210492 , 9.210403 , 9.210129 , 9.210626 , 9.210937 , 9.209695 ,
        9.209389 , 9.209899 , 9.211318 , 9.210599 , 9.20992  , 9.210228 ,
        9.209592 , 9.210546 , 9.210445 , 9.210182 , 9.210332 , 9.21042  ,
        9.21049  , 9.209469 ],
       [9.210407 , 9.210326 , 9.210538 , 9.209883 , 9.210195 , 9.210849 ,
        9.210624 , 9.210434 , 9.211298 , 9.210871 , 9.210429 , 9.2098055,
        9.209641 , 9.211348 , 9.210133 , 9.210434 , 9.209779 , 9.209777 ,
        9.210688 , 9.211195 ],
       [9.210274 , 9.210293 , 9.210471 , 9.209431 , 9.209695 , 9.211036 ,
        9.210258 , 9.21067  , 9.21048  , 9.211063 , 9.210605 , 9.211066 ,
        9.212833 , 9.209844 , 9.211525 , 9.209489 , 9.211074 , 9.2111635,
        9.210392 , 9.209861 ],
       [9.210586 , 9.210237 , 9.210672 , 9.210447 , 9.209808 , 9.20993  ,
        9.210831 , 9.210403 , 9.210555 , 9.209806 , 9.209949 , 9.210496 ,
        9.210847 , 9.211244 , 9.210229 , 9.

In [51]:
loss[0]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([9.210492, 9.210403, 9.210129, 9.210626, 9.210937, 9.209695,
       9.209389, 9.209899, 9.211318, 9.210599, 9.20992 , 9.210228,
       9.209592, 9.210546, 9.210445, 9.210182, 9.210332, 9.21042 ,
       9.21049 , 9.209469], dtype=float32)>

In [52]:
cost=tf.reduce_sum(loss/batch_size)
cost

<tf.Tensor: shape=(), dtype=float32, numpy=184.20626831054688>

In [130]:
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=max_grad_norm)
# lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
#     boundaries=[1000, 2000],
#     values=[0.1, 0.01, 0.001]
# )

# lr = tf.Variable(0.0, trainable=False)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=max_grad_norm)


In [88]:
model=tf.keras.Sequential()
model.add(embedding_layer)
model.add(layer)
model.add(dense)
model.add(activation)
model.compile(loss=crossentropy,optimizer=optimizer)
model.summary()

model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_vocab (Embedding)     │ (None, 20, 200)        │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rnn (RNN)                       │ (None, 20, 128)        │       665,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20, 10000)      │     1,290,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 20, 10000)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,955,088 (15.09 MB)

 Trainable params: 3,955,088 (15.09 MB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_vocab (Embedding)     │ (None, 20, 200)        │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rnn (RNN)                       │ (None, 20, 128)        │       665,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20, 10000)      │     1,290,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 20, 10000)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,955,088 (15.09 MB)

 Trainable params: 3,955,088 (15.09 MB)

 Non-trainable params: 0 (0.00 B)

In [89]:
tvars=model.trainable_variables

In [124]:
tvars=model.trainable_variables
[v.name for v in tvars]

['embeddings',
 'kernel',
 'recurrent_kernel',
 'bias',
 'kernel',
 'recurrent_kernel',
 'bias',
 'kernel',
 'bias']

In [133]:
_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=max_grad_norm)


In [154]:
class PTBModel(object):
    def __init__(self):
        self.batch_size=batch_size
        self.num_steps=num_steps
        self.hidden_size_l1=hidden_size_l1
        self.hidden_size_l2=hidden_size_l2
        self.vocab_size=vocab_size
        self.embedding_vector_size=embedding_vector_size
        self._lr=1.0

        self.model=tf.keras.models.Sequential()
        # self._embedding_layer = tf.keras.layers.Embedding(self.vocab_size, self.embedding_vector_size,batch_input_shape=(self.batch_size, self.num_steps),trainable=True,name="embedding_vocab")  #[10000x200]
        self._embedding_layer=tf.keras.layers.Embedding(self.vocab_size,self.embedding_vector_size,input_shape=(self.num_steps,),trainable=True,name='embedding_vocab')
        self.model.add(self._embedding_layer)

        lstm_cell_l1=tf.keras.layers.LSTMCell(hidden_size_l1)
        lstm_cell_l2=tf.keras.layers.LSTMCell(hidden_size_l2)
        stacked_lstm=tf.keras.layers.StackedRNNCells([lstm_cell_l1,lstm_cell_l2])


        self._RNNlayer=tf.keras.layers.RNN(stacked_lstm,[batch_size,num_steps])

        self._initial_state=tf.Variable(tf.zeros([batch_size,embedding_vector_size]))
        self._RNNlayer.initial_state=self._initial_state

        self.model.add(self._RNNlayer)

        self._dense=tf.keras.layers.Dense(self.vocab_size)
        self.model.add(self._dense)


        self._activation=tf.keras.layers.Activation('softmax')
        self.model.add(self._activation)

        self._optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=max_grad_norm)


        self.model.compile(loss=self.crossentropy,optimizer=self._optimizer)
        self.model.summary()

    def crossentropy(self,y_true,y_pred):
        return tf.keras.losses.sparse_categorical_crossentropy(y_true,y_pred)
    
    def train_batch(self,_input_data,targets):
        self._lr=tf.Variable(0.0,trainable=False)
        tvars=self.model.trainable_variables
        with tf.GradientTape() as tape:
            output_words_prob=self.model(_input_data)
            loss=self.crossentropy(_targets,output_words_prob)
            cost=tf.reduce_sum(loss/self.batch_size)
        grad_t_list=tape.gradient(cost,tvars)
        grads,_=tf.clip_by_global_norm(grad_t_list,max_grad_norm)
        train_op=self._optimizer.apply_gradients(zip(grads,tvars))
        return cost

    def test_batch(self,_input_data,_targets):
        output_words_prob=self.model(_input_data)
        loss=self.crossentropy(_targets,output_words_prob)
        cost=tf.reduce_sum(loss/self.batch_size)
        return cost

    @classmethod
    def instance(cls):
        return PTBModel()


In [167]:
 def run_one_epoch(m,data,is_training=True,verbose=False):
        epoch_size=((len(data)//m.batch_size)-1)//m.num_steps
        start_time=time.time()
        cost=0
        iters=0

        #m.model.reset_states()

        for step,(x,y) in enumerate(reader_ibm.ptb_iterator(data,m.batch_size,m.num_steps)):
            if is_training:
                loss=m.train_batch(x,y)
            
            else:
                loss=m.test_batch(x,y)
        
            cost+=loss
            iters+=m.num_steps

            if verbose and step% (epoch_size//10)==10:
                print(step,epoch_size,np.exp(cost/iters),iters*m.batch_size/(time.time()-start_time))
                
        return np.exp(cost/iters)


In [168]:
raw_data=reader_ibm.ptb_raw_data(data_dir)

In [169]:
train_data,valid_data,test_data,_,_=raw_data

In [170]:
import time 
start = time.time()

# Instantiates the PTBModel class
m=PTBModel.instance()   
K = tf.keras.backend 
max_epoch=2
for i in range(max_epoch):
    print(m)
    print('hello')
    # Define the decay for this epoch
    lr_decay = decay ** max(i - max_epoch_decay_lr, 0.0)
    dcr = learning_rate * lr_decay
    m._lr = dcr
    #K.set_value(m.model.optimizer.learning_rate,m._lr)
    print("Epoch %d : Learning rate: %.3f" % (i + 1, m.model.optimizer.learning_rate))
    # Run the loop for this epoch in the training mode
    train_perplexity = run_one_epoch(m, train_data,is_training=True,verbose=True)
    print("Epoch %d : Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
    # Run the loop for this epoch in the validation mode
    valid_perplexity = run_one_epoch(m, valid_data,is_training=False,verbose=False)
    print("Epoch %d : Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
    
# Run the loop in the testing mode to see how effective was our training
test_perplexity = run_one_epoch(m, test_data,is_training=False,verbose=False)
print("Test Perplexity: %.3f" % test_perplexity)

end = time.time()

print("Total time taken to train and test the model %d seconds", end-start)

C:\Users\kusha\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_vocab (Embedding)     │ (None, 20, 200)        │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rnn_17 (RNN)                    │ (None, 20, 128)        │       665,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 20, 10000)      │     1,290,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_17 (Activation)      │ (None, 20, 10000)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,955,088 (15.09 MB)

 Trainable params: 3,955,088 (15.09 MB)

 Non-trainable params: 0 (0.00 B)

hello
Epoch 1 : Learning rate: 0.010
10 1549 9869.85 1423.6916148819003
164 1549 5703.4175 1340.446807041278
318 1549 1902.6273 1353.7161744783134
472 1549 994.1664 1366.3523644137276
626 1549 686.5838 1390.7192817720793
780 1549 537.0457 1411.640602479027
934 1549 448.02814 1425.2438137108875
1088 1549 388.40402 1436.674644730727
1242 1549 345.35538 1442.8334769188018
1396 1549 312.67905 1448.7809838723101
Epoch 1 : Train Perplexity: 287.236
Epoch 1 : Valid Perplexity: 6518.131
hello
Epoch 2 : Learning rate: 0.010
10 1549 127.64448 1495.7896964178303
164 1549 124.20392 1500.558177547804
318 1549 121.04283 1500.0715889685625
472 1549 118.10506 172.43397552317597


KeyboardInterrupt: 

In [ ]:
impo